1. Setup and Path Utilities (Graph Representation and Operations)
We define the basic structure for the graph and a helper class to manage open paths, which keeps track of the path vertices and its origin (child of the current node) to determine if it is "unrelated" for the combination step

In [12]:
import collections

# --- 1. Path Utilities (Setup and Path Representation) ---

class OpenPath:
    """
    Represents an 'open path' from a subtree.
    
    The path's last vertex (vertices[-1]) is the 'open' end that connects to the parent.
    The 'child_origin' is the child node (u) of the current node (v) from which 
    the path was returned; this is crucial for finding 'unrelated' paths.
    """
    def __init__(self, vertices, child_origin):
        # The list of vertices in the path.
        self.vertices = vertices
        # The vertex connected to the parent.
        self.open_endpoint = vertices[-1] if vertices else None
        # The child of the current node v from which this path originated.
        self.child_origin = child_origin
        
    def __repr__(self):
        return f"Path({self.vertices}, from: {self.child_origin})"

class Tree:
    """Represents the tree using an adjacency list."""
    def __init__(self, edges):
        self.adj = collections.defaultdict(list)
        self.all_vertices = set()
        for u, v in edges:
            self.adj[u].append(v)
            self.adj[v].append(u)
            self.all_vertices.add(u)
            self.all_vertices.add(v)
    
    def get_neighbors(self, v):
        return self.adj[v]
    
    def get_all_vertices(self):
        return list(self.all_vertices)

# --- 2. Path Operations ---

def combine(p1: OpenPath, p2: OpenPath, v) -> list:
    """
    Combines two unrelated open paths p1 and p2 at vertex v, $comb(p_1, p_2)$,
    to form a new closed path: (p1.vertices) + [v] + (p2.vertices reversed).
    """
    closed_path = p1.vertices + [v] + p2.vertices[::-1]
    return closed_path

def concatenate(paths: list[OpenPath], v) -> list[OpenPath]:
    """
    Concatenates a set of paths P with vertex v, $\mathcal{P} \cdot v$,
    to extend them towards the parent.
    """
    extended_paths = []
    for p in paths:
        # Concatenate v to the path. New open end is v.
        new_vertices = p.vertices + [v]
        extended_paths.append(OpenPath(new_vertices, p.child_origin))
    return extended_paths

def find_unrelated_pair(P: list[OpenPath]) -> tuple[OpenPath, OpenPath] or None:
    """
    Finds two paths that came from different child components, i.e., different
    child_origins. It modifies P by removing the two selected paths.
    """
    paths_by_origin = collections.defaultdict(list)
    for p in P:
        paths_by_origin[p.child_origin].append(p)
        
    origins = list(paths_by_origin.keys())
    
    if len(origins) >= 2:
        # Take one path from the first origin and one from the second.
        p1 = paths_by_origin[origins[0]].pop(0)
        p2 = paths_by_origin[origins[1]].pop(0)
        
        # Reconstruct P with the remaining paths (this modifies P in place)
        P.clear()
        
        # Add remaining paths from origins[0] and origins[1]
        P.extend(paths_by_origin[origins[0]])
        P.extend(paths_by_origin[origins[1]])
        
        # Add paths from all other origins
        for i in range(2, len(origins)):
            P.extend(paths_by_origin[origins[i]])
        
        # Return the two selected paths
        return p1, p2
    
    return None

# --- 3. The Core Algorithm: PC(T, v, S) ---

def path_cover_tree_dfs(tree_obj: Tree, v, parent, root, S: list) -> list[OpenPath]:
    """
    Implements the recursive Algorithm 1: PC(T, v, S).
    """
    
    # 2-6: Recursively call for each child u of v
    P_v = []
    children = []
    
    for u in tree_obj.get_neighbors(v):
        if u != parent: 
            children.append(u)
            
            # 3-4: Recursively call PC(T, u, S)
            P_c_open = path_cover_tree_dfs(tree_obj, u, v, root, S)
            
            # 10: $\mathcal{P}_v \leftarrow \cup \mathcal{P}_c^{open}$ 
            for p in P_c_open:
                p.child_origin = u # Set/update the child origin to 'u'
                P_v.append(p)

    P_v_close = [] # 11: $\mathcal{P}_{v}^{close}\leftarrow\emptyset$
    P_v_open = []
    
    # 7-8: If v is a leaf (and not the root of the whole tree)
    if len(children) == 0 and v != root: 
        # $\mathcal{P}_{v}^{open}\leftarrow\{(v,v)\}$ (Singleton path)
        P_v_open.append(OpenPath([v], child_origin=v))
    
    # 9: else (v is an internal node or the root)
    else:
        # 12-16: While $|\mathcal{P}_v|>2$: Combine unrelated paths and add to $\mathcal{P}_v^{close}$.
        while len(P_v) >= 2:
            pair = find_unrelated_pair(P_v)
            
            if pair is None: 
                break 
                
            p1, p2 = pair
            
            # 14: $\mathcal{P}_{v}^{close}\leftarrow\mathcal{P}_{v}^{close}\cup comb(p_{i},p_{j})$
            P_v_close.append(combine(p1, p2, v))
            
        
        # 17: if v is the root of T: Close any remaining paths.
        if v == root:
            if len(P_v) == 2:
                # 18: $\mathcal{P}_{v}^{close}\leftarrow\mathcal{P}_{v}^{close}\cup comb(p_{1},p_{2})$
                p1 = P_v.pop(0)
                p2 = P_v.pop(0)
                P_v_close.append(combine(p1, p2, v))
            elif len(P_v) == 1:
                # 20: $\mathcal{P}_{v}^{close}\leftarrow\mathcal{P}_{v}^{close}\cup p_{1}\cdot v$
                p1 = P_v.pop(0)
                extended_path = concatenate([p1], v)
                P_v_close.append(extended_path[0].vertices)
            
        # 21: else (v is not the root of T and $|\mathcal{P}_v|\le 2$): Pass up remaining paths.
        elif v != root and len(P_v) <= 2: 
            # 22: $\mathcal{P}_{v}^{open}\leftarrow\mathcal{P}_{v}\cdot v$ (Concatenate v to remaining paths).
            P_v_open = concatenate(P_v, v)

    # 26: $S\leftarrow S\cup\mathcal{P}_{v}^{close}$
    S.extend(P_v_close)
    
    # 27: return $\mathcal{P}_{v}^{open}$
    return P_v_open

# --- 4. Main Function and Examples ---

def find_path_cover_on_tree(edges: list) -> tuple[list, int, int]:
    """
    Main function to find the minimum path cover on a tree.
    Returns: (list of path lists, final size, expected minimum size)
    """
    tree_obj = Tree(edges)
    vertices = tree_obj.get_all_vertices()
    
    if not vertices:
        return [], 0, 0
    if len(vertices) == 1:
        return [[vertices[0]]], 1, 1

    # Choose a root (preferably an internal vertex).
    root = vertices[0]
    for v in vertices:
        if len(tree_obj.get_neighbors(v)) > 1:
            root = v
            break 
            
    S_closed_paths = []
    
    # Start the DFS from the chosen root.
    path_cover_tree_dfs(tree_obj, root, parent=None, root=root, S=S_closed_paths)
    
    # Calculate the number of leaves 'l' for the unrooted tree for verification.
    leaves = [v for v in vertices if len(tree_obj.get_neighbors(v)) == 1]
    expected_size = (len(leaves) + 1) // 2
    
    return S_closed_paths, len(S_closed_paths), expected_size

# --- Example Usage ---

# Example 1: Star Graph (l=4, Expected size: 2)
star_graph_edges = [
    (0, 1), (0, 2), (0, 3), (0, 4) 
]
paths, size, expected = find_path_cover_on_tree(star_graph_edges)
print("--- Path Cover on Star Graph Example ---")
print(f"Edges: {star_graph_edges}")
print(f"Expected Size: {expected}, Actual Size: {size}")
print("Paths:", paths)


# Example 2: Path Graph P4 (l=2, Expected size: 1)
path_graph_edges = [
    (0, 1),
    (1, 2),
    (2, 3)
]
paths_p4, size_p4, expected_p4 = find_path_cover_on_tree(path_graph_edges)
print("\n--- Path Cover on P4 Graph Example ---")
print(f"Edges: {path_graph_edges}")
print(f"Expected Size: {expected_p4}, Actual Size: {size_p4}")
print("Paths:", paths_p4)

--- Path Cover on Star Graph Example ---
Edges: [(0, 1), (0, 2), (0, 3), (0, 4)]
Expected Size: 2, Actual Size: 2
Paths: [[1, 0, 2], [3, 0, 4]]

--- Path Cover on P4 Graph Example ---
Edges: [(0, 1), (1, 2), (2, 3)]
Expected Size: 1, Actual Size: 1
Paths: [[0, 1, 2, 3]]


<>:52: SyntaxWarning: invalid escape sequence '\m'
<>:52: SyntaxWarning: invalid escape sequence '\m'
/var/folders/2m/lklfl4ps30jb2xmlyw4htpxh0000gn/T/ipykernel_41100/2526508439.py:52: SyntaxWarning: invalid escape sequence '\m'
  """


The main logic is implemented in a recursive DFS function, mirroring Algorithm 110. The function computes the path cover for the subtree rooted at $v$ and returns the set of open paths $\mathcal{P}_v^{open}$ that must be extended to $v$'s parent.

# references
https://arxiv.org/pdf/2511.07160